# qbacktester Quickstart Guide

This notebook demonstrates the core features of qbacktester, a quantitative backtesting library for financial strategies.

## What You'll Learn
- Loading real market data (SPY 2015-2025)
- Running a moving average crossover strategy
- Analyzing performance metrics
- Visualizing results with professional plots
- Parameter optimization with grid search
- Understanding look-ahead bias and vectorization

Let's get started!


## Setup and Imports

First, we'll set up the Python path to import from our local source directory and load the necessary libraries.


In [ ]:
import sys
import os
from pathlib import Path

# Add src directory to Python path
notebook_dir = Path.cwd()
src_dir = notebook_dir.parent / "src"
sys.path.insert(0, str(src_dir))

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# qbacktester imports
from qbacktester import (
    DataLoader, StrategyParams, run_crossover_backtest, 
    print_backtest_report, grid_search, plot_equity, 
    plot_drawdown, plot_price_signals
)

# Set up plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Imports successful!")
print(f"📁 Source directory: {src_dir}")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")


## 1. Loading Market Data

We'll load 10 years of SPY (S&P 500 ETF) data from 2015 to 2025. The DataLoader automatically handles caching, so subsequent runs will be much faster.


In [ ]:
# Initialize data loader
data_loader = DataLoader()

# Load SPY data from 2015 to 2025
print("📈 Loading SPY data (2015-2025)...")
price_df = data_loader.get_price_data(
    symbol="SPY",
    start="2015-01-01",
    end="2025-01-01",
    interval="1d"
)

print(f"✅ Loaded {len(price_df)} trading days")
print(f"📅 Date range: {price_df.index[0].strftime('%Y-%m-%d')} to {price_df.index[-1].strftime('%Y-%m-%d')}")
print(f"💰 Price range: ${price_df['Close'].min():.2f} - ${price_df['Close'].max():.2f}")

# Display first few rows
print("\n📊 Sample data:")
display(price_df.head())


## 2. Running a Moving Average Crossover Strategy

Now we'll run a classic 20/50 moving average crossover strategy. This strategy:
- **Enters long** when the 20-day SMA crosses above the 50-day SMA
- **Exits long** when the 20-day SMA crosses below the 50-day SMA
- **Avoids look-ahead bias** by entering on the day AFTER the crossover signal

### Why This Matters: Look-Ahead Bias Avoidance

Look-ahead bias is one of the most common mistakes in backtesting. It occurs when you use information that wouldn't have been available at the time of the trade. Our strategy avoids this by:

1. **Signal Generation**: We detect crossovers using only historical data up to that point
2. **Entry Timing**: We enter positions on the day AFTER the crossover, not the same day
3. **Price Execution**: We use the next day's open price (or close if open is unavailable)

This ensures our backtest results are realistic and tradeable in real markets.


In [ ]:
# Define strategy parameters
params = StrategyParams(
    symbol="SPY",
    start="2015-01-01",
    end="2025-01-01",
    fast_window=20,      # 20-day moving average
    slow_window=50,      # 50-day moving average
    initial_cash=100000, # $100,000 starting capital
    fee_bps=5.0,         # 0.05% transaction fee
    slippage_bps=2.5     # 0.025% slippage
)

print("🚀 Running 20/50 Moving Average Crossover Strategy...")
print(f"📊 Parameters: Fast={params.fast_window}, Slow={params.slow_window}")
print(f"💰 Initial Capital: ${params.initial_cash:,}")
print(f"💸 Transaction Costs: {params.fee_bps + params.slippage_bps:.1f} bps total")

# Run the backtest
results = run_crossover_backtest(params)

print("\n✅ Backtest completed successfully!")
print(f"📈 Final Equity: ${results['equity_curve'].iloc[-1]:,.2f}")
print(f"📊 Total Trades: {results['metrics']['num_trades']}")


## 3. Performance Metrics Analysis

Let's examine the detailed performance metrics to understand how our strategy performed.


In [ ]:
# Display comprehensive metrics table
print_backtest_report(results, title="SPY 20/50 Crossover Strategy Results")

# Extract key metrics for further analysis
metrics = results['metrics']
equity_curve = results['equity_curve']
trades = results['trades']

print(f"\n📊 Strategy Summary:")
print(f"   • Total Return: {metrics['total_return']:.2%}")
print(f"   • Annualized Return (CAGR): {metrics['cagr']:.2%}")
print(f"   • Sharpe Ratio: {metrics['sharpe_ratio']:.3f}")
print(f"   • Maximum Drawdown: {metrics['max_drawdown']:.2%}")
print(f"   • Calmar Ratio: {metrics['calmar_ratio']:.3f}")
print(f"   • Hit Rate: {metrics['hit_rate']:.1%}")
print(f"   • Average Win/Loss: {metrics['avg_win_loss']:.2f}")
print(f"   • Total Transaction Costs: ${metrics['total_transaction_costs']:,.2f}")


## 4. Visualizing Results

Now let's create professional visualizations to better understand our strategy's performance.


In [ ]:
# Create a comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('SPY 20/50 Moving Average Crossover Strategy Analysis', fontsize=16, fontweight='bold')

# 1. Price and Signals
ax1 = axes[0, 0]
ax1.plot(price_df.index, price_df['Close'], label='SPY Close', linewidth=1, alpha=0.8)

# Add moving averages
from qbacktester.indicators import sma
sma_20 = sma(price_df, 20, 'Close')
sma_50 = sma(price_df, 50, 'Close')
ax1.plot(price_df.index, sma_20, label='20-day SMA', linewidth=1.5, alpha=0.8)
ax1.plot(price_df.index, sma_50, label='50-day SMA', linewidth=1.5, alpha=0.8)

# Highlight buy/sell signals
signals = results['equity_curve'].index
buy_signals = signals[results['equity_curve'].diff() > 0]
sell_signals = signals[results['equity_curve'].diff() < 0]

if len(buy_signals) > 0:
    ax1.scatter(buy_signals, price_df.loc[buy_signals, 'Close'], 
               color='green', marker='^', s=50, label='Buy Signal', alpha=0.7)
if len(sell_signals) > 0:
    ax1.scatter(sell_signals, price_df.loc[sell_signals, 'Close'], 
               color='red', marker='v', s=50, label='Sell Signal', alpha=0.7)

ax1.set_title('Price Chart with Moving Averages and Signals')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Equity Curve
ax2 = axes[0, 1]
ax2.plot(equity_curve.index, equity_curve, label='Portfolio Value', linewidth=2, color='#2E86AB')
ax2.axhline(y=params.initial_cash, color='gray', linestyle='--', alpha=0.7, label='Initial Capital')
ax2.set_title('Portfolio Equity Curve')
ax2.set_ylabel('Portfolio Value ($)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# 3. Drawdown Chart
ax3 = axes[1, 0]
peak = equity_curve.expanding(min_periods=1).max()
drawdown = (equity_curve - peak) / peak
ax3.fill_between(drawdown.index, drawdown, 0, color='#F24236', alpha=0.6, label='Drawdown')
ax3.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax3.set_title('Portfolio Drawdown')
ax3.set_ylabel('Drawdown (%)')
ax3.set_xlabel('Date')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.1%}'))

# 4. Rolling Sharpe Ratio
ax4 = axes[1, 1]
returns = equity_curve.pct_change().dropna()
rolling_sharpe = returns.rolling(window=252).mean() / returns.rolling(window=252).std() * np.sqrt(252)
ax4.plot(rolling_sharpe.index, rolling_sharpe, label='Rolling Sharpe (252d)', linewidth=1.5, color='#8B5CF6')
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax4.axhline(y=1, color='green', linestyle='--', alpha=0.7, label='Sharpe = 1')
ax4.set_title('Rolling Sharpe Ratio')
ax4.set_ylabel('Sharpe Ratio')
ax4.set_xlabel('Date')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Format x-axis for all subplots
for ax in axes.flat:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.xaxis.set_major_locator(mdates.YearLocator())
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()

print("📊 Visualizations created successfully!")


## 5. Parameter Optimization with Grid Search

Let's explore how different parameter combinations affect performance. We'll run a grid search to find the optimal fast/slow window combinations.

### Understanding Vectorization

qbacktester is built with vectorization as a core principle. This means:

1. **NumPy Operations**: All technical indicators use vectorized NumPy operations
2. **Pandas Efficiency**: Signal generation and portfolio calculations use efficient pandas operations
3. **No Python Loops**: The core algorithms avoid explicit Python for-loops (except where sequential processing is required by the nature of portfolio management)
4. **Parallel Processing**: Grid search can utilize multiple CPU cores for faster optimization

This design allows us to run hundreds of backtests in seconds, making parameter optimization practical and efficient.


In [ ]:
# Define parameter grids for optimization
fast_grid = [5, 10, 15, 20, 25, 30]
slow_grid = [40, 50, 60, 80, 100, 120]

print("🔍 Running parameter optimization...")
print(f"⚡ Fast windows: {fast_grid}")
print(f"🐌 Slow windows: {slow_grid}")
print(f"🎯 Total combinations: {len(fast_grid) * len(slow_grid)}")

# Run grid search (this may take a minute or two)
optimization_results = grid_search(
    symbol="SPY",
    start="2015-01-01",
    end="2025-01-01",
    fast_grid=fast_grid,
    slow_grid=slow_grid,
    metric="sharpe",
    initial_cash=100000,
    fee_bps=5.0,
    slippage_bps=2.5,
    n_jobs=4,  # Use 4 parallel workers
    verbose=True
)

print("\n✅ Optimization completed!")
print(f"📊 Found {len(optimization_results)} valid parameter combinations")

# Display top 5 results
print("\n🏆 Top 5 Parameter Combinations:")
top_results = optimization_results.head()
for i, (_, row) in enumerate(top_results.iterrows(), 1):
    print(f"{i}. Fast={row['fast']:2d}, Slow={row['slow']:3d} | "
          f"Sharpe={row['sharpe']:.3f} | CAGR={row['cagr']:.2%} | MaxDD={row['max_dd']:.2%}")


## 6. Optimization Results Heatmap

Let's visualize the optimization results as a heatmap to see how different parameter combinations perform.


In [ ]:
# Create pivot table for heatmap
pivot_data = optimization_results.pivot_table(
    values='sharpe', 
    index='slow', 
    columns='fast', 
    aggfunc='mean'
)

# Create heatmap
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Sharpe Ratio Heatmap
im1 = ax1.imshow(pivot_data.values, cmap='RdYlGn', aspect='auto', vmin=0, vmax=pivot_data.values.max())
ax1.set_title('Sharpe Ratio Heatmap', fontsize=14, fontweight='bold')
ax1.set_xlabel('Fast Window')
ax1.set_ylabel('Slow Window')
ax1.set_xticks(range(len(pivot_data.columns)))
ax1.set_xticklabels(pivot_data.columns)
ax1.set_yticks(range(len(pivot_data.index)))
ax1.set_yticklabels(pivot_data.index)

# Add colorbar
cbar1 = plt.colorbar(im1, ax=ax1)
cbar1.set_label('Sharpe Ratio')

# Add text annotations
for i in range(len(pivot_data.index)):
    for j in range(len(pivot_data.columns)):
        value = pivot_data.iloc[i, j]
        if not np.isnan(value):
            ax1.text(j, i, f'{value:.2f}', ha='center', va='center', 
                    color='white' if value < pivot_data.values.max()/2 else 'black', fontsize=8)

# CAGR Heatmap
pivot_cagr = optimization_results.pivot_table(
    values='cagr', 
    index='slow', 
    columns='fast', 
    aggfunc='mean'
)

im2 = ax2.imshow(pivot_cagr.values, cmap='RdYlGn', aspect='auto', vmin=0, vmax=pivot_cagr.values.max())
ax2.set_title('CAGR Heatmap', fontsize=14, fontweight='bold')
ax2.set_xlabel('Fast Window')
ax2.set_ylabel('Slow Window')
ax2.set_xticks(range(len(pivot_cagr.columns)))
ax2.set_xticklabels(pivot_cagr.columns)
ax2.set_yticks(range(len(pivot_cagr.index)))
ax2.set_yticklabels(pivot_cagr.index)

# Add colorbar
cbar2 = plt.colorbar(im2, ax=ax2)
cbar2.set_label('CAGR')

# Add text annotations
for i in range(len(pivot_cagr.index)):
    for j in range(len(pivot_cagr.columns)):
        value = pivot_cagr.iloc[i, j]
        if not np.isnan(value):
            ax2.text(j, i, f'{value:.1%}', ha='center', va='center', 
                    color='white' if value < pivot_cagr.values.max()/2 else 'black', fontsize=8)

plt.tight_layout()
plt.show()

# Find and display the best parameters
best_params = optimization_results.iloc[0]
print(f"\n🏆 Best Parameters Found:")
print(f"   • Fast Window: {best_params['fast']}")
print(f"   • Slow Window: {best_params['slow']}")
print(f"   • Sharpe Ratio: {best_params['sharpe']:.3f}")
print(f"   • CAGR: {best_params['cagr']:.2%}")
print(f"   • Max Drawdown: {best_params['max_dd']:.2%}")
print(f"   • Final Equity: ${best_params['equity_final']:,.2f}")


## 7. Running the Optimized Strategy

Let's run the backtest with the best parameters we found to see how it performs.


In [ ]:
# Create optimized strategy parameters
optimized_params = StrategyParams(
    symbol="SPY",
    start="2015-01-01",
    end="2025-01-01",
    fast_window=int(best_params['fast']),
    slow_window=int(best_params['slow']),
    initial_cash=100000,
    fee_bps=5.0,
    slippage_bps=2.5
)

print(f"🚀 Running optimized strategy: {optimized_params.fast_window}/{optimized_params.slow_window} crossover...")

# Run the optimized backtest
optimized_results = run_crossover_backtest(optimized_params)

# Display results
print_backtest_report(optimized_results, title="Optimized Strategy Results")

# Compare with original strategy
print(f"\n📊 Strategy Comparison:")
print(f"{'Metric':<20} {'Original (20/50)':<15} {'Optimized':<15} {'Improvement':<15}")
print("-" * 65)

original_metrics = results['metrics']
optimized_metrics = optimized_results['metrics']

comparisons = [
    ('Sharpe Ratio', f"{original_metrics['sharpe_ratio']:.3f}", f"{optimized_metrics['sharpe_ratio']:.3f}", 
     f"{optimized_metrics['sharpe_ratio'] - original_metrics['sharpe_ratio']:+.3f}"),
    ('CAGR', f"{original_metrics['cagr']:.2%}", f"{optimized_metrics['cagr']:.2%}", 
     f"{optimized_metrics['cagr'] - original_metrics['cagr']:+.2%}"),
    ('Max Drawdown', f"{original_metrics['max_drawdown']:.2%}", f"{optimized_metrics['max_drawdown']:.2%}", 
     f"{optimized_metrics['max_drawdown'] - original_metrics['max_drawdown']:+.2%}"),
    ('Final Equity', f"${original_metrics['final_equity']:,.0f}", f"${optimized_metrics['final_equity']:,.0f}", 
     f"${optimized_metrics['final_equity'] - original_metrics['final_equity']:+,.0f}")
]

for metric, orig, opt, improvement in comparisons:
    print(f"{metric:<20} {orig:<15} {opt:<15} {improvement:<15}")


## 8. Key Takeaways and Best Practices

### What We've Learned

1. **Look-Ahead Bias Prevention**: Our strategy correctly avoids look-ahead bias by entering positions on the day after signal generation, using realistic execution prices.

2. **Vectorization Benefits**: The entire backtesting process is vectorized, allowing us to run hundreds of parameter combinations in seconds.

3. **Transaction Costs Matter**: Even small transaction costs (7.5 bps total) can significantly impact strategy performance.

4. **Parameter Optimization**: Grid search revealed that different parameter combinations can have vastly different performance characteristics.

### Best Practices for Quantitative Backtesting

1. **Always Avoid Look-Ahead Bias**: Never use future information in your trading decisions
2. **Include Transaction Costs**: Real trading involves fees and slippage
3. **Use Vectorized Operations**: Leverage NumPy and Pandas for performance
4. **Test Multiple Parameters**: Don't rely on a single parameter set
5. **Validate on Out-of-Sample Data**: Use walk-forward analysis for robust validation
6. **Consider Market Regimes**: Strategies may perform differently in different market conditions

### Next Steps

To continue exploring qbacktester:

- Try different strategies (RSI, MACD, Bollinger Bands)
- Experiment with walk-forward analysis
- Add more sophisticated risk management
- Explore different asset classes and timeframes

Happy backtesting! 📈


In [ ]:
# Final summary
print("🎉 Quickstart Guide Complete!")
print("\n📚 What we accomplished:")
print("   ✅ Loaded 10 years of SPY data")
print("   ✅ Ran a 20/50 moving average crossover strategy")
print("   ✅ Analyzed comprehensive performance metrics")
print("   ✅ Created professional visualizations")
print("   ✅ Optimized parameters with grid search")
print("   ✅ Compared original vs optimized strategies")
print("   ✅ Learned about look-ahead bias and vectorization")

print("\n🚀 Ready to build your own strategies with qbacktester!")
